# Data cleaning

### Imports
Import libraries and write settings here.

In [1]:
%load_ext autoreload
%autoreload
%load_ext dotenv
%dotenv

import pandas as pd
import os
import qgrid
import ipywidgets
import numpy as np

In [2]:
filepath = "data/25100029_MetaData-dirty.df"
df_dictionary = pd.read_pickle('data/25100029_MetaData-dirty.df')
df = pd.read_pickle('data/25100029-dirty.df')
qgrid.enable()

## Guide to StatsCan datasets
See https://www.statcan.gc.ca/eng/developers/csv/user-guide for description of column names.  Re-summarized here for reference:

Comments on columns:

|Column | StatsCan Notes | Our Notes |
|:--| :--| :--|
|UOM ID (Units of measure ID) | | seems only to be necessarily to have a unique identifier to deal with Eng/French translation of units of measurement, so ignore this for now|
|SCALAR_FACTOR | The scalar factor associated with a data series, displayed as text. | Disregard this if all entries are 'units' (scalar factor of 1)|
| Scalar_ID | | See above |
| Vector |Unique variable length reference code time-series identifier, consisting of the letter 'V', followed by up to 10 digits. (i.e. V1234567890, V1, etc.) | Don't understand this | 
| Status | Shows various states of a data value using symbols. These symbols are described in the [symbol legend](https://www.statcan.gc.ca/eng/concepts/definitions/guide-symbol) and notes contained in the metadata file. Some symbols accompany a data value while others replace a data value. i.e. – A, B, C, D, E, F,.., X, 0s | It's at row 98 of the data dictionary here|
| Symbol | Describes data points that are preliminary or revised, displayed using the symbols p and r. These symbols accompany a data value.| Can drop this | 
| DECIMALS | This field displays the decimal precision for a given value. | We should append this to VALUE before converting it to numeric |






In [3]:
symbol_legend = df_dictionary.iloc[98:113].copy()
col_rename_dict = {"Cube Title": "Description", "Product Id": "Symbol"}
symbol_legend = symbol_legend.rename(columns=col_rename_dict)
symbol_legend = symbol_legend[col_rename_dict.values()]
symbol_legend.reset_index(drop=True) # Otherwise, index starts at 98

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Rename and drop columns

In [4]:
def columns_with_unique_val(df):
    unique = []
    val = []
    for col in df.columns:
        values = df[col].unique()
        if len(values) == 1:
            unique.append(col)
            val.append(values[0])
            
    return unique, val

# Drop columns whose only value is nan
df.dropna(axis=1, how='all')

column_map={"REF_DATE": "Year", "GEO": "Region", "UOM": "Unit of measurement"}
df = df.rename(columns=column_map)

# Unless we combine multiple tables where these values differ 
# (for instance, multiple units of measurement), let's store this outside of the df
drop_cols, vals = columns_with_unique_val(df)
table_metadata = {drop_cols[i]: vals[i] for i in range(len(drop_cols))}

df = df.drop(drop_cols, axis=1)

In [5]:
df['Supply and demand characteristics'].unique()
df.columns
# table_metadata
df

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Set datatypes

In [6]:
display(df.dtypes)

df['Year'] = pd.to_numeric(df['Year'])
df['VALUE'] = pd.to_numeric(df['VALUE'])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Exploration

In [7]:
def search(term, df):
    mask = np.column_stack([df[col].str.contains(term, na=False) for col in df])
    return df.loc[mask.any(axis=1)]
    
search("symbol", df_dictionary)

NameError: name 'np' is not defined